In [6]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [7]:
H_LIST = np.array([0, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 5000, 10000, 20000])
A_LIST = np.array([340.29, 340.10, 339.91, 339.53, 339.14, 338.76, 338.38, 337.98, 337.6, 337.21, 336.82, 336.43, 320.54, 299.53, 295.07])

In [8]:
MACH_LIST = np.array([0.4, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6,
                     1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7,
                     2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6])

CX43_LIST = np.array([0.157, 0.158, 0.158, 0.160, 0.190, 0.325, 0.378, 0.385, 0.381, 0.371,
                    0.361, 0.351, 0.342, 0.332, 0.324, 0.316, 0.309, 0.303, 0.297,
                    0.292, 0.287, 0.283, 0.279, 0.277, 0.273, 0.270, 0.267, 0.265,
                    0.263, 0.263, 0.261, 0.260])

In [46]:
class ExtBalRS:

    def __init__(self, q, d, cx_list, mach_list, rho0=1.225, k=2, h=1e-3):

        self.q = q
        self.S = np.pi * 0.25 * d * d
        self.cx_list = cx_list
        self.mach_list = mach_list
        self.rho0 = rho0
        self.k = k
        self.h = h

    def cx(self, v, y):
        a_zv = np.interp(y, H_LIST, A_LIST)
        M = v / a_zv
        cx = np.interp(M, self.mach_list, self.cx_list)
        return cx

    def dcx_dv(self, v, y):
        dv = v * self.h
        return (self.cx(v+dv, y) - self.cx(v-dv, y)) / (2*dv)

    def dcx_dy(self, v, y):
        if y == 0:
            return 0
        dy = y * self.h
        return (self.cx(v, y+dy) - self.cx(v, y-dy)) / (2*dy)

    def H(self, y):
        return ((20000 - y) / (20000 + y))

    def dH_dy(self, y):
        return -(1/(y+20000))-(20000-y)/(y+20000)**2


    def get_i(self, Pv, v, y):
        rho = self.rho0 * self.H(y)
        S = self.S
        q = self.q
        k = self.k
        return -0.5 * (Pv * S * k**2 * v**2 * self.cx(v, y) * rho)/q

    def external_bal_rs(self, t, y):
        p = y[4:]
        y = y[:4]
        Px, Py, Pv, Ptheta = p
        
        S = self.S
        q = self.q
        k = self.k
        rho0 = self.rho0
        rho = rho0 * self.H(y)
        
        g = 9.80665
        # dy = np.zeros_like(y)
        # dp = np.zeros_like(p)

        # Расчет производных вектора состояния
        x, y, v, theta = y
        i = self.get_i(Pv, v, y)
        
        cx = self.cx(v, y)
        dx = v * np.cos(theta)  # Расчет приращения координаты X
        dy = v * np.sin(theta)  # Расчет приращения координаты Y
        Fx = -0.5 * i * cx * rho * v * v  # Расчет силы лобового сопротивления
        dv = (Fx * S - q * g * np.sin(theta)) / q  # Расчет приращения скорости V
        dtheta = -g * np.cos(theta) / v  # Расчет приращения угла teta

        # Расчет производных вектора сопряженных переменных

        dpv = Pv * (0.5 * S * i * v**2 * self.dcx_dv(v, y) * rho + S * i * v * self.cx(v, y) * rho) / q
        dpv += Py * np.sin(theta)
        dpv += Ptheta * g * np.cos(theta) / v / v
        dpv += Px * np.cos(theta)

        dpx = 0

        dpy = Pv * (0.5 * S * i * v**2 * self.cx(v, y) * self.dH_dy(y) * rho0 + 0.5 * S * i * v**2 * self.dcx_dy(v, y) * rho) / q

        dptheta = -(Px * v * np.sin(theta)) + Ptheta * g * np.sin(theta) / v + Py * v * np.cos(theta) - Pv * g * np.cos(theta)
        
        return [dx, dy, dv, dtheta, Px, Py, Pv, Ptheta]

In [47]:
bal_rs = ExtBalRS(43, 0.1524, CX43_LIST, MACH_LIST)

In [48]:
v0 = 950
x0 = 0
y0 = 0
theta0 = np.deg2rad(45)
Px = Py = Pv = Ptheta = 3

In [49]:
state0 = [x0, y0, v0, theta0, Px, Py, Pv, Ptheta]

In [50]:
hit_ground = lambda t, y, *args: y[1]
hit_ground.terminal = True
hit_ground.direction = -1

In [51]:
sol = solve_ivp(bal_rs.external_bal_rs,
                (0., 200),
                state0,
                dense_output=True,
                t_eval=np.linspace(0, 200, 200),
                events=hit_ground
               )

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.